In [1]:
from test_case_agent import testcase_agent
from optimization_agent import optimization_suggestor_agent

## Input

In [2]:
code = """def calculate_factorials_up_to_number(n):
    if n < 0:
        raise ValueError("Input must be a non-negative integer.")

    factorials = [1]  # Initialize with the factorial of 0

    factorial = 1
    for i in range(1, n + 1):
        factorial *= i
        factorials.append(factorial)

    return factorials"""

## Initialize Testcase Agent

In [3]:
agent = testcase_agent(code = code,
                    #    test_case_list =["calculate_factorials_up_to_number(100000)"]
                       )

In [4]:
agent.gen_test_case()

'Worst case\n```python\ncalculate_factorials_up_to_number(1000)\n```\n\nMedium case\n```python\ncalculate_factorials_up_to_number(10)\n```\n\nBest case\n```python\ncalculate_factorials_up_to_number(0)\n```'

In [5]:
agent.extract_test_cases()

calculate_factorials_up_to_number(1000)
--------
calculate_factorials_up_to_number(10)
--------
calculate_factorials_up_to_number(0)
--------


In [6]:
agent.join_test_case()

['import time\nimport sys\nimport os\nimport psutil\n\ndef memory_usage_psutil():\n    # return the memory usage in MB\n    process = psutil.Process(os.getpid())\n    mem_info = process.memory_info()\n    return mem_info.rss\n\ndef test_case():\n    start_time = time.time()\n    start_memory = memory_usage_psutil()\n\n    # Call the function here\n    calculate_factorials_up_to_number(1000)\n    \n    end_time = time.time()\n    end_memory = memory_usage_psutil()\n\n    print(f"Time taken: {end_time - start_time} seconds")\n    print(f"Memory used: {end_memory - start_memory} B")\n\ndef calculate_factorials_up_to_number(n):\n    if n < 0:\n        raise ValueError("Input must be a non-negative integer.")\n\n    factorials = [1]  # Initialize with the factorial of 0\n\n    factorial = 1\n    for i in range(1, n + 1):\n        factorial *= i\n        factorials.append(factorial)\n\n    return factorials\ntest_case()',
 'import time\nimport sys\nimport os\nimport psutil\n\ndef memory_usag

In [7]:
agent.run_test_case()

['Time taken: 0.0006549358367919922 seconds\nMemory used: 524288 B\n',
 'Time taken: 0.0001819133758544922 seconds\nMemory used: 0 B\n',
 'Time taken: 0.00013327598571777344 seconds\nMemory used: 0 B\n']

In [8]:
vas = agent.extract_info_tg()

In [9]:
print(vas)

[{'Time taken': 0.0006549358367919922, 'Memory used': 524288.0}, {'Time taken': 0.0001819133758544922, 'Memory used': 0.0}, {'Time taken': 0.00013327598571777344, 'Memory used': 0.0}]


In [10]:
print(*vas, sep = "\n")

{'Time taken': 0.0006549358367919922, 'Memory used': 524288.0}
{'Time taken': 0.0001819133758544922, 'Memory used': 0.0}
{'Time taken': 0.00013327598571777344, 'Memory used': 0.0}


------

## Optimize Code

In [11]:
op_agent = optimization_suggestor_agent(code= code,
                                        execution_obj= vas)

In [12]:
op_agent.raw_optimizations()

'1. Use List Comprehension: The code can be optimized by using list comprehension to generate the factorials in a more concise and efficient manner. Instead of using a for loop and appending to the "factorials" list, we can calculate each factorial directly using a list comprehension.\n\n2. Remove Redundant Assignment: The variable "factorial" is initialized to 1 and then immediately multiplied by "i". We can remove the separate initialization step and simply start the loop at 2 instead of 1.\n\n3. Error Handling: The code currently assumes that the input "n" is a non-negative integer. It may be a good idea to include error handling and validation to ensure that the input meets the required conditions.\n\n4. Return as Generator: Instead of returning a complete list of factorials, returning a generator using yield can save memory. A generator allows values to be generated on the fly instead of storing them all in memory at once.\n\nWith these optimizations, the code can be made more eff

In [13]:
response = op_agent.optimizer()

In [14]:
print(response)

```python
def calculate_factorials_up_to_number(n):
    if not isinstance(n, int) or n < 0:
        raise ValueError("Input must be a non-negative integer.")

    return (factorial := 1, yield factorial)[0] if n == 0 else (factorial := 1, yield factorial)[0] and (factorial := factorial * i for i in range(2, n + 1))
```


In [15]:
extracted_blocks = op_agent.extract_code()

def calculate_factorials_up_to_number(n):
    if not isinstance(n, int) or n < 0:
        raise ValueError("Input must be a non-negative integer.")

    return (factorial := 1, yield factorial)[0] if n == 0 else (factorial := 1, yield factorial)[0] and (factorial := factorial * i for i in range(2, n + 1))
--------


In [16]:
print(extracted_blocks)

def calculate_factorials_up_to_number(n):
    if not isinstance(n, int) or n < 0:
        raise ValueError("Input must be a non-negative integer.")

    return (factorial := 1, yield factorial)[0] if n == 0 else (factorial := 1, yield factorial)[0] and (factorial := factorial * i for i in range(2, n + 1))


------------------

In [23]:
extracted_blocks = """def calculate_factorials_up_to_number(n):
    if not isinstance(n, int) or n < 0:
        raise ValueError("Input must be a non-negative integer.")

    factorials = {}

    def factorial(k):
        if k in factorials:
            return factorials[k]
        elif k == 0:
            return 1
        else:
            result = k * factorial(k - 1)
            factorials[k] = result
            return result

    return [factorial(i) for i in range(n + 1)]"""

## Optimized Code Testcases

In [24]:
op_code_test = testcase_agent(
    code = extracted_blocks)

In [25]:
op_code_test.gen_test_case()
op_code_test.extract_test_cases()
op_code_test.join_test_case()
op_code_test.run_test_case()
vas = op_code_test.extract_info_tg()

calculate_factorials_up_to_number(1000)
--------
calculate_factorials_up_to_number(10)
--------
calculate_factorials_up_to_number(0)
--------


In [26]:
print(vas)

[{'Time taken': 0.0008258819580078125, 'Memory used': 655360.0}, {'Time taken': 0.00017380714416503906, 'Memory used': 0.0}, {'Time taken': 0.00018930435180664062, 'Memory used': 0.0}]


----

In [36]:
from line_profiler import LineProfiler
import io
import sys

lp = LineProfiler()

def profile_print(func_to_profile, *func_args):

    lp.add_function(func_to_profile)
    lp.runctx('func_to_profile(*func_args)', locals=locals(), globals=globals())

    stdout_ = sys.stdout  #Keep track of the previous value.
    stream = io.StringIO()
    sys.stdout = stream  # alternative output

    lp.print_stats(output_unit=0.001)        # values are in milliseconds
    sys.stdout = stdout_  # restore the previous stdout.

    output = stream.getvalue().split('\n')
    print(output)
    for line in output:
        print(line)

numbers = [1, 2, 3, 4, 5]

def do_stuff(numbers):
    s = sum(numbers)
    prod = 1
    for n in numbers:
        prod *= n
    return s, prod

profile_print(do_stuff, numbers)

['Timer unit: 0.001 s', '', 'Total time: 6.57e-06 s', 'File: /tmp/ipykernel_22549/1248220270.py', 'Function: do_stuff at line 26', '', 'Line #      Hits         Time  Per Hit   % Time  Line Contents', '==============================================================', '    26                                           def do_stuff(numbers):', '    27         1          0.0      0.0     28.3      s = sum(numbers)', '    28         1          0.0      0.0      4.4      prod = 1', '    29         6          0.0      0.0     34.2      for n in numbers:', '    30         5          0.0      0.0     27.9          prod *= n', '    31         1          0.0      0.0      5.1      return s, prod', '', '']
Timer unit: 0.001 s

Total time: 6.57e-06 s
File: /tmp/ipykernel_22549/1248220270.py
Function: do_stuff at line 26

Line #      Hits         Time  Per Hit   % Time  Line Contents
    26                                           def do_stuff(numbers):
    27         1          0.0      0.0     28.

In [56]:
from line_profiler import LineProfiler
import io
import sys

def profile_print(func_to_profile, *func_args):
    lp = LineProfiler()
    lp.add_function(func_to_profile)
    lp.enable_by_count()
    lp.runctx('func_to_profile(*func_args)', locals=locals(), globals=globals())

    stdout_ = sys.stdout  # keep track of the previous value.
    stream = io.StringIO()
    sys.stdout = stream  # redirect output to the stream
    lp.print_stats(output_unit=0.001)  # values are in milliseconds
    sys.stdout = stdout_  # restore the previous stdout.

    profile_output = stream.getvalue().split('\n')
    for line in profile_output:
        print(line)
    code_time_dict = {}
    for line in profile_output:
        cols = line.split()
        # Only process complete lines
        if len(cols) >= 6:
            hit_str = cols[0]
            time_perc_str = cols[4]
            code_str = " ".join(cols[5:])
            # Exclude any empty/whitespace lines or profiling metadata
            if hit_str.isdigit() and time_perc_str.replace('.', '', 1).isdigit():
                code_time_dict[code_str] = {"hits": hit_str, "percent_time": time_perc_str}
    return code_time_dict

numbers = [1, 2, 3, 4, 5]

def do_stuff(numbers):
    s = sum(numbers)
    prod = 1
    for n in numbers:
        prod *= n
    return s, prod

result_dict = profile_print(do_stuff, numbers)
from pprint import pprint
pprint(result_dict)

Timer unit: 0.001 s

Total time: 3.938e-06 s
File: /tmp/ipykernel_22549/2113319959.py
Function: do_stuff at line 35

Line #      Hits         Time  Per Hit   % Time  Line Contents
    35                                           def do_stuff(numbers):
    36         1          0.0      0.0     31.0      s = sum(numbers)
    37         1          0.0      0.0      4.2      prod = 1
    38         6          0.0      0.0     29.5      for n in numbers:
    39         5          0.0      0.0     29.8          prod *= n
    40         1          0.0      0.0      5.6      return s, prod


{'for n in numbers:': {'hits': '38', 'percent_time': '29.5'},
 'prod *= n': {'hits': '39', 'percent_time': '29.8'},
 'prod = 1': {'hits': '37', 'percent_time': '4.2'},
 'return s, prod': {'hits': '40', 'percent_time': '5.6'},
 's = sum(numbers)': {'hits': '36', 'percent_time': '31.0'}}


In [1]:
from profiler_agent import profiler

In [2]:
code = """def do_stuff(numbers):
    s = sum(numbers)
    prod = 1
    for n in numbers:
        prod *= n
    return s, prod"""
testcase = "do_stuff([1, 2, 3, 4, 5])"

In [3]:
p_agent = profiler(
    code= code,
    testcase= testcase
                   )

In [4]:
p_agent.initialize_profiler()

from line_profiler import LineProfiler
import io
import sys
import json
import ast

def profile_print(func_to_profile, *func_args):
    lp = LineProfiler()
    lp.add_function(func_to_profile)
    lp.enable_by_count()
    lp.runctx('func_to_profile(*func_args)', locals=locals(), globals=globals())

    stdout_ = sys.stdout  # keep track of the previous value.
    stream = io.StringIO()
    sys.stdout = stream  # redirect output to the stream
    lp.print_stats(output_unit=0.001)  # values are in milliseconds
    sys.stdout = stdout_  # restore the previous stdout.

    profile_output = stream.getvalue().split('\n')
    for line in profile_output:
        print(line)
    code_time_dict = {}
    for line in profile_output:
        cols = line.split()
        # Only process complete lines
        if len(cols) >= 6:
            hit_str = cols[0]
            time_perc_str = cols[4]
            code_str = " ".join(cols[5:])
            # Exclude any empty/whitespace lines or profiling metada